Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [200]:
import numpy as np
import cv2

# Repeatability
np.random.seed(0)

#VFILENAME =  "walking.mp4"
VFILENAME = "Mario-N-Friends.mp4" 
ESC_KEY = 27

In [201]:
def get_video_data(filename):
    video = cv2.VideoCapture(filename)
    fps = round(video.get(cv2.CAP_PROP_FPS))
    framerate = round(1000 / fps) # 1000 ms / FPS
    resolution = (round(video.get(cv2.CAP_PROP_FRAME_WIDTH)),round(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    video.release()
    print(framerate)
    print(fps)
    print(resolution)
    video.release()
    return framerate, fps, resolution

In [202]:
FRAMERATE, FPS, RESOLUTION = get_video_data(VFILENAME)
WIDTH, HEIGHT = RESOLUTION

42
24
(1188, 720)


Load video frames from file

In [203]:
def get_frames(filename):
    video = cv2.VideoCapture(filename) # Open Video
    while video.isOpened():
        retrieved, frame = video.read() # Read frames
        if retrieved:
            yield frame 
        else:
            break
    video.release() # Video is closed, release resources
    yield None

Creating a particle cloud

In [204]:
NUM_PARTICLES = 3000
VEL_RANGE = 0.5
# particles = [particle][(x,y)][velocityX][velocityY]
def initialize_particles():
    particles = np.random.rand(NUM_PARTICLES,4) # 5000 Rows, 4 Columns, Number between 0 and 1
    particles = particles * np.array((WIDTH,HEIGHT,VEL_RANGE,VEL_RANGE)) # Range of X, Y, and velocities for every particle
    particles[:,2:4] -= VEL_RANGE/2.0 # Center Velocity at 0
    print(particles[:20,:])
    return particles

Moving particles according to their velocity state

In [205]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [206]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        particles[i,0] = max(0,min(WIDTH - 1, particles[i,0])) # Prevents x value from being greater than WIDTH
        particles[i,1] = max(0,min(HEIGHT - 1, particles[i,1])) # Prevents y value from being greater than HEIGHT 
    return particles

Measure each particle's quality

In [207]:
#TARGET_COLOR = np.array((156,74,38)) # BGR Color Value
TARGET_COLOR = np.array((0,0,255))
def compute_errors(particles, frame):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y = int(particles[i,1])
        pixel_color = frame[y,x,:] # y,x, BGR
        errors[i] = np.sum((TARGET_COLOR - pixel_color) ** 2) # Difference in color values
    return errors

Assign weights to the particles based on their quality of match

In [208]:
def compute_weights(errors):
    weights = np.max(errors) - errors # Invert errors (Highest error = Lowest Weight)
    # Set edge particles weight to 0
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH-1) | 
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT - 1)
    ] = 0.0
    weights = weights**3 # Higher weight exponential, Stricter Color Tracking
    return weights

Resample particles according to their weights

In [209]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights) # Normalize
    
    # New particles based on weights (High Weight = High Chance)
    index_numbers = np.random.choice(
        NUM_PARTICLES,
        size = NUM_PARTICLES,
        p=probabilities)
    particles = particles[index_numbers,:] # Replace particles with chosen indices
    
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    return particles, (int(x),int(y))

Fuzz the particles

In [210]:
POS_SIGMA = 1.0 * 15
VEL_SIGMA = 0.5 * 5
def apply_noise(particles):
    noise = np.concatenate(
    (
        np.random.normal(0.0,POS_SIGMA,(NUM_PARTICLES,1)), # X-position
        np.random.normal(0.0,POS_SIGMA,(NUM_PARTICLES,1)), # Y-position
        np.random.normal(0.0,VEL_SIGMA,(NUM_PARTICLES,1)), # X-Velocity
        np.random.normal(0.0,VEL_SIGMA,(NUM_PARTICLES,1)), # Y-Velocity
    ),
    axis=1)
    particles += noise
    return particles

Display the video frames

In [211]:
def display(frame, particles, location):
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0), 1)
    if len(location) > 0:
        cv2.circle(frame, location, 15, (0,0,255),5)
    cv2.imshow('frame',frame)
    if cv2.waitKey(30) == ESC_KEY: # Pause Video
        if cv2.waitKey(0) == ESC_KEY: # Exit Video
            return True
            
    return False

Main routine

In [212]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()


[[ 6.51990443e+02  5.14936344e+02  5.13816880e-02  2.24415915e-02]
 [ 5.03301902e+02  4.65043761e+02 -3.12063944e-02  1.95886500e-01]
 [ 1.14483136e+03  2.76077894e+02  1.45862519e-01  1.44474599e-02]
 [ 6.74836939e+02  6.66429580e+02 -2.14481971e-01 -2.06435350e-01]
 [ 2.40194562e+01  5.99486289e+02  1.39078375e-01  1.85006074e-01]
 [ 1.16259859e+03  5.75394166e+02 -1.92603189e-02  1.40264588e-01]
 [ 1.40510018e+02  4.60743135e+02 -1.78323356e-01  2.22334459e-01]
 [ 6.19955806e+02  2.98556597e+02 -1.17722194e-01  1.37116845e-01]
 [ 5.41906595e+02  4.09272443e+02 -2.40605100e-01  5.88177485e-02]
 [ 7.27169719e+02  4.44192478e+02  2.21874039e-01  9.09101496e-02]
 [ 4.27095386e+02  3.14663007e+02  9.88155980e-02 -2.19887264e-01]
 [ 7.92118858e+02  4.82859266e+02 -1.44808719e-01 -1.85536851e-01]
 [ 3.74728881e+02  2.61871755e+02  3.50983852e-02 -3.06992433e-02]
 [ 1.17418812e+03  7.34722637e+01 -1.45561622e-01 -1.69345241e-01]
 [ 7.75892691e+02  1.82369954e+02 -1.68446136e-02 -1.27787204e

In [213]:
fourcc = cv2.VideoWriter_fourcc('M','P','4','V') # MP4V Format Output
video_out = cv2.VideoWriter("walking-output.mp4",fourcc,FPS,RESOLUTION) # (filename,fourcc,frames per sec, resolution)

def save_video(frame, particles, location):
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0), 1)
    if len(location) > 0:
        cv2.circle(frame, location, 15, (0,0,255),5)
    video_out.write(frame)
            
    return False

In [214]:
# Save Video to Disk

particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: 
        break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    save_video(frame, particles, location)
    
cv2.destroyAllWindows()
video_out.release()


[[ 2.55239220e+02  5.43844341e+02  1.05200837e-01 -1.47252434e-01]
 [ 3.20433962e+02  4.07919836e+02  1.86683823e-01  4.24144183e-02]
 [ 9.93154669e+02  1.86595056e+02 -2.42621051e-01 -2.05833342e-01]
 [ 3.48054913e+02  1.20164516e+02  2.43096868e-01  7.37804814e-02]
 [ 7.11662466e+02  9.43442652e+01 -1.99136335e-02  2.17086899e-01]
 [ 7.70442960e+01  6.99017293e+02 -1.19268833e-01  2.37838498e-01]
 [ 5.36619760e+01  2.58800923e+02  2.05728964e-01 -7.54415769e-03]
 [ 3.76875416e+02  2.07133173e+02  6.99169020e-03  8.08817024e-02]
 [ 1.17172691e+03  3.73632740e+02  2.45631817e-01 -6.77126653e-02]
 [ 6.86768564e+02  5.29568108e+02 -1.65414903e-01  1.10607881e-01]
 [ 2.25666766e+02  2.05132898e+02 -2.14161413e-01 -2.40155131e-01]
 [ 9.22889544e+02  7.19268127e+02 -4.71333029e-02 -4.55309281e-02]
 [ 2.61137310e+02  4.70415376e+02 -1.40769652e-01 -1.79841219e-01]
 [ 4.53282299e+02  3.93207094e+00 -6.28364205e-02 -2.25690498e-01]
 [ 6.75408080e+02  1.01237278e+02 -2.36757918e-01 -3.84197803e